In [57]:
from random import randint
from gmpy2 import is_square, is_even, jacobi
import math

COMPOSITE = False
PROBABLY_PRIME = True
PERFECT_SQUARE=True
NOT_PERFECT_SQUARE=True

def bit(k, n):
    return n & (1 << k)

class FermatTest:
    def isPrime(n, k = 3):
        # Base case
        if n < 2: return COMPOSITE
        if n == 2: return PROBABLY_PRIME

        # Even number
        if is_even(n): return COMPOSITE

        # Test k rounds
        for i in range(k):
            a = randint(2, n - 2)
            if pow(a, n - 1, n) != 1: return COMPOSITE

        return PROBABLY_PRIME
    
class MillerRabinTest:
    def __isWitness(a, d, n):
        x = pow(a, d, n)

        if x == 1 or x == n - 1:
            return True
        
        while d != n - 1:
            x = (x * x) % n 
            d *= 2

            if x == 1: return False
            if x == n - 1: return True

        return False
    
    def isWitness(a, n):
        d = n - 1
        while not (d % 2):
            d //= 2

        x = pow(a, d, n)

        if x == 1 or x == n - 1:
            return PROBABLY_PRIME
        
        while d != n - 1:
            x = (x * x) % n 
            d *= 2

            if x == 1: return COMPOSITE
            if x == n - 1: return PROBABLY_PRIME

        return COMPOSITE
    
    def isPrime(n, k):
        # Base case
        if n < 2: return COMPOSITE
        if n == 2: return PROBABLY_PRIME

        # Even number
        if is_even(n): return COMPOSITE

        # Find d s.t. n - 1 = 2^s * d
        d = n - 1
        while not (d % 2):
            d //= 2
        
        # Test k rounds
        for i in range(k):
            if not MillerRabinTest.__isWitness(randint(2, n-2), d, n):
                return COMPOSITE
            
        return PROBABLY_PRIME
    
class LucasTest:
    def isPrime(n : int):
        # Base case
        if n < 2: return COMPOSITE
        if n == 2: return PROBABLY_PRIME

        # Even number or Perfect square number
        if is_even(n) or is_square(n): return COMPOSITE

        def alternate():
            value = 5
            while True:
                yield value
                if value > 0:
                    value += 2
                else:
                    value -= 2
                value = -value

        for D in alternate():
            if n in (D, -D): continue
            js = jacobi(D, n)
            if js == 0: return COMPOSITE
            if js == -1: break

        K = n + 1
        r = K.bit_length() - 1

        U_i = V_i = 1
        U_temp = V_temp = 0

        for i in range(r-1, -1, -1):
            U_temp = U_i * V_i % n
            V_temp = (((V_i ** 2 + (U_i ** 2 * D)) * K) >> 1) % n

            if bit(i, K):
                U_i = (((U_temp + V_temp) * K) >> 1) % n
                V_i = (((V_temp + U_temp * D) * K) >> 1) % n
            else:
                U_i = U_temp
                V_i = V_temp
        
        if not U_i:
            return PROBABLY_PRIME
        
        return COMPOSITE

class BPSWTest:
    def isPrime(n):
        if not MillerRabinTest.isWitness(2, n): return COMPOSITE
        return LucasTest.isPrime(n)
    
class PerfectSquare:
    def isSquare(C):
        n=math.ceil (math.log2(C))+1
        m=math.ceil(n/2)
        i=0
        X=[2**(m-1)]
        while(True):
            i=i+1
            X.append((X[i-1]**2 + C)/(2*X[i-1]))
            if(X[i]**2 < 2**m+C):
                break

        if ( C== math.floor(X[i])**2):
            return PERFECT_SQUARE
        else:
            return NOT_PERFECT_SQUARE

class JacobiSymbol:
    # a: any integer
    # n: any integer
    def jacobi(a,n):
        #Reduce a modulo n
        a= a % n

        #Handle special cases
        if a==1 or n==1:
            return 1
        if a==0:
            return 0
        
        #Compute e and a1
        e = 0
        a1 = a
        while a1 % 2 == 0:
            e += 1
            a1 //= 2

        #Compute s
        if e % 2 == 0:
            s = 1
        elif n % 8 == 1 or n % 8 == 7:
            s = 1
        elif n % 8 == 3 or n % 8 == 5:
            s = -1
        
        #Compute the sign adjustment
        if n % 4 == 3 and a1 % 4 == 3:
            s = -s
            
        # Compute n1 and call the function recursively
        n1 = n % a1
        return s * JacobiSymbol.jacobi(n1, a1)

In [27]:
Carmichael_Numbers = [
    561,
    41041,
    825265,
    321197185,
    5394826801,
    232250619601,
    9746347772161,
    1436697831295441,
    60977817398996785,
    7156857700403137441,
    1791562810662585767521,
    87674969936234821377601,
    6553130926752006031481761,
    1590231231043178376951698401,
    35237869211718889547310642241,
    32809426840359564991177172754241,
    2810864562635368426005268142616001,
    349407515342287435050603204719587201,
    125861887849639969847638681038680787361,
    12758106140074522771498516740500829830401,
    2333379336546216408131111533710540349903201,
    294571791067375389885907239089503408618560001,
    130912961974316767723865201454187955056178415601,
    13513093081489380840188651246675032067011140079201,
    7482895937713262392883306949172917048928068129206401,
    1320340354477450170682291329830138947225695029536281601,
    379382381447399527322618466130154668512652910714224209601,
    70416887142533176417390411931483993124120785701395296424001,
    2884167509593581480205474627684686008624483147814647841436801,
    4754868377601046732119933839981363081972014948522510826417784001,
    1334733877147062382486934807105197899496002201113849920496510541601,
    260849323075371835669784094383812120359260783810157225730623388382401,
    112505380450296606970338459629988782604252033209350010888227147338120001,
]

In [31]:
cnt = 0

while not MillerRabinTest.isPrime(Carmichael_Numbers[-1], 3) and cnt < 1e5:
    cnt += 1

print(cnt)

100000


In [66]:
print(PerfectSquare.isSquare(121))

print(JacobiSymbol.jacobi(11,31))


True
-1
